In [2]:
import numpy as np

In [3]:
@np.vectorize
def round_tick(price, tick_size):
    return tick_size * round(price/tick_size)

In [ ]:
class Exchange():

    def __init__(self):
        
        self.products = {}
        self.prices = {}
        self.users = {}
        self.perceived_prices = {}
        self.order_book = {}
        self.ownership_book = {}
    

    def add_product(self, identifier, market_cap, volume, tick_size):
        self.products[identifier] = {"market_cap": market_cap, "volume": volume, "tick_size": tick_size}
        self.order_book[identifier] = {"buy": {}, "sell": {}}
        self.prices[identifier] = round((market_cap/volume)/tick_size) * tick_size #This slightly changes market_cap by rounding the price to the discrete plane
        self.ownership_book[identifier] = {"exchange": volume} #Exchange owns everything

    def add_user(self, user_id, capital):
        self.users[user_id] = {"capital": capital, "usable_capital": capital}

    def transfer_product(self, user_id_1, user_id_2, identifier, trade_volume, direction):
        self.ownership_book[identifier][user_id_1] += {"buy": 1, "sell": -1}[direction]*trade_volume
        self.ownership_book[identifier][user_id_2] += {"buy": 1, "sell": -1}[direction]*trade_volume

    def transfer_capital(self, user_id_1, user_id_2, trade_volume, price, direction):
        self.users[user_id_1]["capital"] += {"buy": 1, "sell": -1}[direction]*trade_volume*price
        self.users[user_id_2]["capital"] += {"buy": 1, "sell": -1}[direction]*trade_volume*price

        #whetver makes money gets an increase to usable capital
        if direction == "sell":
            self.users[user_id_1]["usable_capital"] += {"buy": 1, "sell": -1}[direction]*trade_volume*price
        else:
            self.users[user_id_2]["usable_capital"] += {"buy": 1, "sell": -1}[direction]*trade_volume*price
    

    def add_order(self, user_id, identifier, price, volume):
        direction = "buy" if volume > 0 else "sell"
        if self.order_book[identifier][direction][price] is None:
            self.order_book[identifier][direction][price]["orders"] = []
            self.order_book[identifier][direction][price]["total"] = 0
        self.order_book[identifier][direction][price]["orders"].append((user_id, abs(volume)))
        self.order_book[identifier][direction][price]["total"] += abs(volume)
        
        if direction == "buy":
            self.users["usable_capital"] -= volume*price # users total active orders cannot exceed their capital

        if direction == "buy":
            if sum([self.order_book[identifier]["sell"][key]["total"] for key in self.order_book[identifier]["sell"].keys() if key <= price]) > 0:
                self.add_trade(self, user_id, identifier, price, volume)
            
        if direction == "sell":
            if sum([self.order_book[identifier]["buy"][key]["total"] for key in self.order_book[identifier]["buy"].keys() if key >= price]) > 0:
                self.add_trade(self, user_id, identifier, price, volume)                



    def add_trade(self, user_id_1, identifier, price, volume):#repeat trades in here until filled or cant fill anymore
        direction = "buy" if volume > 0 else "sell"
        while volume > 0:
            if volume > self.order_book[identifier][direction][price][0]:
                user_id_2 = self.order_book[identifier][direction][price][0][0]
                trade_volume = self.order_book[identifier][direction][price][0][1]
                self.order_book[identifier][direction][price].pop(0)
                volume -= trade_volume

                #transfer ownership and adjust capital
                self.transfer_product(self, user_id_1, user_id_2, identifier, trade_volume, direction)
                self.transfer_capital(self, user_id_1, user_id_2, trade_volume, price, direction)


    def open_exchange(self):
        #self.perceived_prices = np.zeros((len(self.users), len(self.products))) + np.array(list(self.prices.values())) + \
        #np.transpose([round_tick(np.random.normal(0, np.sqrt(self.prices[product]), len(self.users))/self.prices[product], self.products[product]["tick_size"]) for product in self.products])
        #self.user_sentiment = (self.perceived_prices - np.zeros((len(self.users), len(self.products))) - np.array(list(self.prices.values())))/np.array(list(self.prices.values()))


        #sentiment = (self.perceived_prices - self.prices)/self.prices

        # for user in self.users:
        #     favourite = 
            
        
        #someone has made an order
        current_identifier = 1
        current_volume = 1
        current_price = 1
        if 




















            

In [7]:
any([1,2,3] > 0)

TypeError: '>' not supported between instances of 'list' and 'int'

In [6]:
sum([1,2,3][:1])

1

In [5]:
sorted([1,3,-1])

[-1, 1, 3]

In [ ]:
#when adding an order, check the opposite side
    #if buy
        #check all prices at or below the order price, counting from the lowest price




In [4]:
test = ["hi", "low"]
hm = test.pop(0)
test
hm

'hi'

In [5]:
exchange = Exchange()
exchange.add_product(identifier = "Apple", market_cap = 10e7, volume = 10e4, tick_size = 1/40)
exchange.add_product(identifier = "Microsoft", market_cap = 2*10e7, volume = 10e4, tick_size = 1/40)
exchange.add_user(user_id = "James", capital = 10e4)
exchange.add_user(user_id = "Bob", capital = 4*10e4)
exchange.add_user(user_id = "Adam", capital = 10e5)
exchange.open_exchange()
#Now we have an exchange, owner of all stocks is exchange, price is initial price, perceived price is initial price
vars(exchange)

{'products': {'Apple': {'market_cap': 100000000.0,
   'volume': 100000.0,
   'tick_size': 0.025},
  'Microsoft': {'market_cap': 200000000.0,
   'volume': 100000.0,
   'tick_size': 0.025}},
 'prices': {'Apple': 1000.0, 'Microsoft': 2000.0},
 'users': {'James': {'capital': 100000.0},
  'Bob': {'capital': 400000.0},
  'Adam': {'capital': 1000000.0}},
 'perceived_prices': array([[ 999.975, 1999.975],
        [1000.   , 2000.025],
        [ 999.975, 2000.025]]),
 'user_sentiment': array([[-2.50e-05, -1.25e-05],
        [ 0.00e+00,  1.25e-05],
        [-2.50e-05,  1.25e-05]])}